go to my favourite tea supplier page: https://yunomi.life/collections/green-tea?page=1

and scrape me all green tea listings (29 pages). You do not need to visit the pages themselves, only get the information from the list of products.

For every tea I want you to save: 
* tea name
* tea origin
* tea price
* number of reviews
* availability
* link to the page of the tea

if you need help, at the botom in **HELP** section, you can find how to do every single of these for one example. Please try it on your own first.

Once it is all done, and you have a list of lits containing all of this info for every single tea, make pandas dataframe out of it, save it.

In [ ]:
!pip install -q bs4

In [ ]:
#Load Packages
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib.request
import requests
print("Packages loaded")
import time
import random

Packages loaded


In [ ]:
page_url = "https://yunomi.life/collections/green-tea?page=1"
request = requests.get(page_url) 
soup = BeautifulSoup(request.content,"html")


tea= soup.find("div", attrs={"class":"product-list"})

tea= tea.find_all("div", attrs={"class":"product-item"})
soup

In [ ]:
page_url = "https://yunomi.life/collections/green-tea?page=1"
request = requests.get(page_url) 
soup = BeautifulSoup(request.content,"html")

tea= soup.find("div", attrs={"class":"product-list"})

tea= tea.find_all("div", attrs={"class":"product-item"})

tea_origin = tea[0].find("a",attrs={"class":"product-item__vendor link"}).text
tea_name = tea[0].find("a",attrs={"class":"product-item__title text--strong link"}).text
tea_star = tea[0].find("span",attrs={"class":"ssw-review-count"}).text
tea_star = tea_star.replace("(","").replace(")","")
tea_price = tea[0].find("span",attrs={"class":"price"}).text
tea_availability =tea[0].find("span",attrs={"class":"product-item__inventory inventory inventory--high"}).text
tea_link = "https://yunomi.life/" + tea[0].find("a",attrs={"class":"product-item__image-wrapper product-item__image-wrapper--with-secondary"})["href"]

print(tea_origin ,"\n",tea_name,"\n",tea_star,"\n",tea_price,"\n",tea_availability,"\n",tea_link)

Hachimanjyu Yakushima Tea 
 Hachimanjyu: Naturally Grown "Hisoka", Spring First Flush Green Tea (Yutaka Midori) 
  10  
 From JPY ¥600 
 Available 
 https://yunomi.life//collections/green-tea/products/hachimanjyu-naturally-grown-hisoka-spring-first-flush-green-tea-yutaka-midori


In [ ]:
!pip install alive-progress

     |████████████████████████████████| 72 kB 3.0 MB/s 
     |████████████████████████████████| 207 kB 71.2 MB/s 
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210096 sha256=e3cc94888a6137a77926c6c8ca8543799ec8e4e9a165168420b5cfc9aa9f8086
  Stored in directory: /root/.cache/pip/wheels/71/fc/c8/142ef03b79c02b618fe80b9f2d01c25bd55f60b0c58eab3f0e
Successfully built grapheme


In [ ]:
class ScrapeYunomi():
    MAX_PAGES = 30
    PAGE_BASE = "https://yunomi.life/"
    
    def __init__(self, continue_scraping=False):
        self.write_new_save_file(continue_scraping)
        self.links_to_scrape = self.get_links_to_scrape()
        self.df_tea = self.scrape_index_page()

    def write_new_save_file(self, continue_scraping):
        if continue_scraping:
            pass
        else:
            with open('/work/data/homework 23/save_tea.csv', 'w') as file_object:
                file_object.write("")

    def get_links_to_scrape(self):
        list_of_links=[]
        for i in range(1, self.MAX_PAGES + 1):
            link = "https://yunomi.life/collections/green-tea?page=" + str(i)
            list_of_links.append(link)
        return list_of_links

    def scrape_index_page(self):
        records=[]
        with open('/work/data/homework 23/save_tea.csv', 'a', encoding='UTF8') as file_object:
            with alive_bar(len(self.links_to_scrape)) as bar:
                for link in self.links_to_scrape:
                    request = requests.get(link)
                    soup = BeautifulSoup(request.content,'html.parser')

                    tea_div= soup.find("div", attrs={"class":"product-list"})
                    list_of_all_teas= tea_div.find_all("div", attrs={"class":"product-item"})

                    if tea_div and list_of_all_teas:
                        for result in list_of_all_teas:  
                            try:
                                tea_origin = result.find("a",attrs={"class":"product-item__vendor link"}).text
                            except:
                                tea_origin = "None"

                            try:
                                tea_star = result.find("span",attrs={"class":"ssw-review-count"}).text
                                tea_star = tea_star.replace("(","").replace(")","")
                            except:
                                tea_star = "None"

                            try:
                                tea_price = result.find("span",attrs={"class":"price"}).text
                            except:
                                tea_price = "None"

                            try:
                                tea_availability =result.find("span",attrs={"class":"product-item__inventory"}).text
                            except:
                                tea_availability = "None"

                            try:
                                tea_name = result.find("a", attrs={"class":"product-item__title text--strong link"}).text
                            except:
                                tea_name = "None"

                            try:
                                tea_link = "https://yunomi.life/" + result.find("a",attrs={"class":"product-item__image-wrapper"})["href"]
                            except:
                                tea_link = "None"
                            file_object.write(f"{tea_name},{tea_origin},{tea_star},{tea_price},{tea_availability},{tea_link}\n")
                            records.append((tea_name,tea_origin,tea_star,tea_price,tea_availability,tea_link))
                    else:
                        print(link)
                    bar()
        col_names=["tea_name","tea_origin","tea_star","tea_price","tea_availability","tea_link"]
        df = pd.DataFrame(records,columns=col_names)
        df.to_csv("/work/data/homework 23/teas.csv")

        return df
            


model = ScrapeYunomi()
# model.links_to_scrape

# pd.read_csv('/work/data/homework 23/save_tea.csv', index_col=None, header=None, error_bad_lines=False, names=["tea_name","tea_origin","tea_star","tea_price","tea_availability","tea_link"])
pd.read_csv('/work/data/homework 23/teas.csv')

on 29: https://yunomi.life/collections/green-tea?page=30
|████████████████████████████████████████| 30/30 [100%] in 47.8s (0.63/s)


In [ ]:
max_pages = 2
list_of_image_urls = []
records = []  
for i in range(1, max_pages+1):
    page_url = "https://yunomi.life/collections/green-tea?page=" + str(i)
    request = requests.get(page_url)
    soup = BeautifulSoup(request.content,'html.parser')

    tea= soup.find("div", attrs={"class":"product-list"})

    tea= tea.find_all("div", attrs={"class":"product-item"})


    for result in tea:  
        tea_name = result.find("a", attrs={"class":"product-item__title text--strong link"}).text
        tea_origin = result.find("a",attrs={"class":"product-item__vendor link"}).text
        tea_star = result.find("span",attrs={"class":"ssw-review-count"}).text
        tea_star = tea_star.replace("(","").replace(")","")
        tea_price = result.find("span",attrs={"class":"price"}).text
        tea_availability =result.find("span",attrs={"class":"product-item__inventory"}).text

        try:
            tea_link = "https://yunomi.life/" + result.find("a",attrs={"class":"product-item__image-wrapper"})["href"]
        except:
            tea_link = None

        records.append((tea_name,tea_origin,tea_star,tea_price,tea_availability,tea_link))

    
col_names=["tea_name","tea_origin","tea_star","tea_price","tea_availability","tea_link"]
df = pd.DataFrame(records,columns=col_names)
df

,tea_name,tea_origin,tea_star,tea_price,tea_availability,tea_link
0,"Hachimanjyu: Naturally Grown ""Hisoka"", Spring ...",Hachimanjyu Yakushima Tea,10,From JPY ¥600,Available,https://yunomi.life//collections/green-tea/pro...
1,Kurihara Tea #01: Takumi 匠 (The Artisan) - Kur...,Kurihara Tea Farm,11,"From JPY ¥1,000",Available,https://yunomi.life//collections/green-tea/pro...
2,"Nakamura-en #01: Gyokuro Mecha, The Discovery ...",Nakamura-en,36,From JPY ¥600,"In stock, 13 units",https://yunomi.life//collections/green-tea/pro...
3,"Yunomi Matcha: Uji no Aji - Kiyomizu, Standard...",Yunomi Tea,19,"From JPY ¥1,999",Available,https://yunomi.life//collections/green-tea/pro...
4,Kurihara Tea #03: Shin 心 (Heart) - Premium Gyo...,Kurihara Tea Farm,12,From JPY ¥999,Available,https://yunomi.life//collections/green-tea/pro...
5,NaturaliTea #19: Mountain-grown Fukamushi Senc...,NaturaliTea,12,From JPY ¥600,Available,https://yunomi.life//collections/green-tea/pro...
6,"Yunomi Matcha: Uji no Aji - Kibune, Imperial C...",Yunomi Tea,14,"From JPY ¥2,999",Available,https://yunomi.life//collections/green-tea/pro...
7,Yunomi Matcha Naturally Grown - Premium Ceremo...,Yunomi Tea,16,"JPY ¥1,999",Available,https://yunomi.life//collections/green-tea/pro...
8,ON SALE! Yunomi House Blend Green Tea 100g - Y...,Yunomi Tea,16,From JPY ¥699,Available,https://yunomi.life//collections/green-tea/pro...
9,Kurihara Tea #04: Mountain-Grown Yamecha Kabus...,Kurihara Tea Farm,11,From JPY ¥600,Available,https://yunomi.life//collections/green-tea/pro...


# help

In [ ]:
# download content of the page
url = "https://yunomi.life/collections/green-tea?page=1"
request = requests.get(url)
soup = BeautifulSoup(request.content,"html")

# find divider that holds all the tea products 
teas = soup.find("div", attrs={"class":"product-list"})

# find ALL tea products
teas = teas.find_all("div", attrs={"class":"product-item"})

# for a tea from the list of all products find price, name, review count, origin
# converting review count to integer
# keeping price as it is, because it might be that there will be price in JPY and USD
tea_price = teas[0].find("span", attrs={"class":"money"}).text
tea_name = teas[0].find("a", attrs={"class":"product-item__title text--strong link"}).text
tea_review_count = teas[0].find("span", attrs={"class": "ssw-review-count"})
tea_review_count = int(tea_review_count.text.strip().replace("(","").replace(")",""))
tea_origin = teas[0].find("a", attrs={"class":"product-item__vendor link"}).text
availability = teas[0].find("span", attrs={"class":"product-item__inventory inventory inventory--high"}).text
tea_link = "https://yunomi.life/" + teas[0].find("a", attrs={"class":"product-item__image-wrapper product-item__image-wrapper--with-secondary"})["href"]

print(tea_name, tea_origin, tea_price, tea_review_count, availability, tea_link)

Hachimanjyu: Naturally Grown "Hisoka", Spring First Flush Green Tea (Yutaka Midori) Hachimanjyu Yakushima Tea JPY ¥600 10 Available https://yunomi.life//collections/green-tea/products/hachimanjyu-naturally-grown-hisoka-spring-first-flush-green-tea-yutaka-midori


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d3ce7c8-a514-49e4-9ba4-a5899ac52ea5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>